In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

# Load the dataset
# Assuming the dataset is stored in 'cnc_data.csv' with features and a target column named 'failure'
data = pd.read_csv('cnc_data.csv')

# Separate features and target
X = data.drop('failure', axis=1)
y = data['failure']

# Split the dataset into training and testing sets using stratification
# Stratification ensures that the rare failure events are proportionally represented in both sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Define the models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'SVM': SVC(probability=True, random_state=42)  # Enable probability estimates for ROC AUC
}

# Dictionary to store evaluation results for each model
results = {}

# Loop over the models, train them, and compute evaluation metrics
for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Compute evaluation metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)  # Recall is critical as it minimizes false negatives
    f1 = f1_score(y_test, y_pred)
    
    # Calculate ROC AUC if probability estimates or decision function is available
    if hasattr(model, 'predict_proba'):
        y_probs = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_probs)
    else:
        try:
            y_scores = model.decision_function(X_test)
            roc_auc = roc_auc_score(y_test, y_scores)
        except:
            roc_auc = None
    
    # Store the computed metrics
    results[name] = {
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1,
        'ROC AUC': roc_auc
    }

# Convert the results dictionary to a DataFrame for easier comparison
results_df = pd.DataFrame(results).T

# Since recall is the priority in this imbalanced scenario, sort the results by Recall
results_df.sort_values(by='Recall', ascending=False, inplace=True)

# Display the summary results
results_df